# [【SOTA】マイナビ × SIGNATE Student Cup 2019: 賃貸物件の家賃予測](https://signate.jp/competitions/264)

## データ読み込み

In [431]:
import pandas as pd
import numpy as np
import pathlib
import os

# 学習データ、テストデータの読み込み
train_path = pathlib.Path("./DATA/train.csv")
test_path = pathlib.Path("./DATA/test.csv")

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [472]:
test_data.head()

,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間,予測結果
0,31471,東京都世田谷区深沢５丁目1-27,東急大井町線\t等々力駅\t徒歩15分\t\t東急田園都市線\t桜新町駅\t徒歩24分\t\...,2LDK,49年0ヶ月,南,50.22m2,8階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座／...,ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\tL字キッチン,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\tエレベーター\t公営水...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【コンビニ】 74m\t【スーパー】 215m\t【病院】 313m\t【小学校】 219m...,鉄骨造,NaN,158098
1,31472,東京都目黒区八雲１丁目11-8,東急東横線\t都立大学駅\t徒歩6分\t\t東急東横線\t自由が丘駅\t徒歩17分\t\t東...,1R,0年2ヶ月,南東,20.88m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCSアンテナ／\tBSアンテナ／\tインターネット使用料無料,エアコン付\tシューズボックス／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水／...,"駐車場\t近隣\t34,000円(税込)\t距離200m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 128m\t【スーパー】 313m\t【コンビニ】 374m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,80236
2,31473,東京都豊島区池袋本町２丁目22-2,東武東上線\t北池袋駅\t徒歩10分\t\t埼京線\t板橋駅\t徒歩16分\t\t山手線\t...,1K,23年4ヶ月,南東,26.93m2,1階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t2面採光／\t室内洗濯...,"駐車場\t空有\t5,000円(税込)\t(敷金あり\t1ヶ月)\t駐輪場\t空有\tバイク...",【スーパー】 529m\t【スーパー】 810m\t【コンビニ】 316m\t【コンビニ】 ...,鉄骨造,2年間,94671
3,31474,東京都杉並区和泉１丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩10分\t\t京王線\t代田橋駅\t徒歩8分...,1K,36年2ヶ月,南東,23.57m2,1階／2階建,シャワー,ガスコンロ／\t給湯,NaN,エアコン付\tシューズボックス／\t2面採光／\t室外洗濯機置場\t公営水道／\t下水／\t...,駐輪場\t空有\t0円,【スーパー】 286m\t【スーパー】 702m\t【コンビニ】 489m\t【コンビニ】 ...,木造,2年間,69022
4,31475,東京都杉並区堀ノ内２丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩12分\t\t丸ノ内線(池袋－荻窪)\t新高...,2LDK,45年4ヶ月,南,50m2,4階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別,コンロ設置可（口数不明）／\t給湯,インターネット対応,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置...,"駐車場\t近隣\t22,600円\t距離177m",【コンビニ】 312m\t【コンビニ】 436m\t【スーパー】 902m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,133378


## 前処理

In [433]:
# 簡単な前処理
train_data_1 = train_data[["id", "賃料", "間取り", "面積", "所在階"]]
test_data_1 = test_data[["id", "間取り", "面積", "所在階"]]

In [434]:
# 間取りにlabel encodingを適用
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.concat([train_data_1["間取り"],test_data_1["間取り"]]))

train_data_1["間取りID"] = le.transform(train_data_1["間取り"])
test_data_1["間取りID"] = le.transform(test_data_1["間取り"])

<ipython-input-434-95a8caebc059>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_1["間取りID"] = le.transform(train_data_1["間取り"])
<ipython-input-434-95a8caebc059>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_1["間取りID"] = le.transform(test_data_1["間取り"])


In [435]:
# 面積のカラムタイトルを面積[m2]に置換
train_data_1 = train_data_1.rename(columns={"面積": "面積m2"})
test_data_1 = test_data_1.rename(columns={"面積": "面積m2"})

In [436]:
# カラムタイトル面積[m2]の要素からm2を削除
train_data_1['面積m2'] = train_data_1['面積m2'].str.replace('m2', '').astype(float)
test_data_1['面積m2'] = test_data_1['面積m2'].str.replace('m2', '').astype(float)

In [437]:
# 所在階の中身を"／"で2つの列に分割
train_data_1 = pd.concat([train_data_1, train_data_1['所在階'].str.split('／', expand=True)], axis=1)
test_data_1 = pd.concat([test_data_1, test_data_1['所在階'].str.split('／', expand=True)], axis=1)

In [438]:
# 分割した所在階のカラム名変更　0:所在、1:階層
train_data_1 = train_data_1.rename(columns={0:"所在", 1:"階層"})
test_data_1 = test_data_1.rename(columns={0:"所在", 1:"階層"})

In [439]:
# 所在と階層の要素を整形
train_data_1['所在'] = train_data_1['所在'].str.replace('階', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('建', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('地下', '-')
train_data_1['所在'] = train_data_1['所在'].str.replace('\（.*\）', '', regex=True)
train_data_1['階層'] = train_data_1['階層'].str.replace('階建', '')
train_data_1['階層'] = train_data_1['階層'].str.replace('\（.*\）', '', regex=True)
train_data_1['所在'] = train_data_1['所在'].replace('', np.nan)

test_data_1['所在'] = test_data_1['所在'].str.replace('階', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('建', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('地下', '-')
test_data_1['所在'] = test_data_1['所在'].str.replace('\（.*\）', '', regex=True)
test_data_1['階層'] = test_data_1['階層'].str.replace('階建', '')
test_data_1['階層'] = test_data_1['階層'].str.replace('\（.*\）', '', regex=True)
test_data_1['所在'] = test_data_1['所在'].replace('', np.nan)

In [440]:
# "階層"がNoneの箇所を"所在"の値で埋める
train_data_1['階層'].fillna(train_data_1['所在'], inplace=True)
test_data_1['階層'].fillna(test_data_1['所在'], inplace=True)

In [441]:
# 所在と階層の要素をfloat型に変換する
train_data_1['所在'] = train_data_1['所在'].astype(float)
train_data_1['階層'] = train_data_1['階層'].astype(float)

test_data_1['所在'] = test_data_1['所在'].astype(float)
test_data_1['階層'] = test_data_1['階層'].astype(float)

In [442]:
# 間取りと所在階のカラムを削除
train_data_1 = train_data_1.drop('間取り', axis=1)
train_data_1 = train_data_1.drop('所在階', axis=1)

test_data_1 = test_data_1.drop('間取り', axis=1)
test_data_1 = test_data_1.drop('所在階', axis=1)

In [443]:
# IDの削除
train_data_1_no_ID = train_data_1.drop(["id"], axis=1)
test_data_1_no_ID = test_data_1.drop(["id"], axis=1)

## データチェック

In [444]:
train_data_1.head()

,id,賃料,面積m2,間取りID,所在,階層
0,1,75000,20.01,3,1.0,12.0
1,2,76000,16.50,9,5.0,10.0
2,3,110000,22.05,3,12.0,15.0
3,4,150000,60.48,14,3.0,4.0
4,5,74000,39.66,10,1.0,2.0


In [445]:
test_data_1.head()

,id,面積m2,間取りID,所在,階層
0,31471,50.22,14,8.0,8.0
1,31472,20.88,9,3.0,4.0
2,31473,26.93,3,1.0,4.0
3,31474,23.57,3,1.0,2.0
4,31475,50.00,14,4.0,4.0


In [446]:
train_data_1.isnull().sum()

id         0
賃料         0
面積m2       0
間取りID      0
所在       120
階層         0
dtype: int64

In [447]:
test_data_1.isnull().sum()

id         0
面積m2       0
間取りID      0
所在       114
階層         1
dtype: int64

## 学習 

In [448]:
# 特徴データと目的変数の設定
train_x = train_data_1_no_ID.drop(["賃料"], axis=1)
train_y = train_data_1_no_ID["賃料"]

test_x = test_data_1_no_ID

In [449]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

scores = []
# GBDT用のハイパーパラメータ
params = {"objective": "reg:squarederror", "sileng":1, "random_state":71}
num_round = 50

# クロスバリデーション
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx, in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # GBDTで学習実行
    dtrain = xgb.DMatrix(tr_x, label=tr_y) # enable_categorical=True
    dvalid = xgb.DMatrix(va_x, label=va_y)
    dtest = xgb.DMatrix(test_x)
    
    watchlist = [(dtrain, "train"), (dvalid, "eval")]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)

[16:36:29] WARNING: ..\src\learner.cc:541: 
Parameters: { sileng } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:101464.46875	eval-rmse:101134.56250
[1]	train-rmse:75651.82812	eval-rmse:75958.37500
[2]	train-rmse:58470.76172	eval-rmse:59684.37109
[3]	train-rmse:47378.18750	eval-rmse:49406.23438
[4]	train-rmse:40417.51172	eval-rmse:43100.24219
[5]	train-rmse:36214.20703	eval-rmse:39222.83203
[6]	train-rmse:33776.40234	eval-rmse:37142.20703
[7]	train-rmse:32268.12891	eval-rmse:35887.89844
[8]	train-rmse:31364.70312	eval-rmse:35211.92578
[9]	train-rmse:30845.23633	eval-rmse:34737.82812
[10]	train-rmse:30480.51367	eval-rmse:34433.15625
[11]	train-rmse:30155.82617	eval-rmse:34314.83594
[12]	train-rmse:29985.79883	eval-rmse:34239.74609
[13]	train-rmse:29750.81055	eval

## 予測

In [450]:
# テストデータでの予測
pred = model.predict(dtest)

In [451]:
pred

array([158098.73 ,  80236.01 ,  94671.695, ..., 176716.45 , 262251.72 ,
        84474.33 ], dtype=float32)

In [452]:
len(pred)

31262

In [453]:
len(test_data_1)

31262

## データ出力

In [458]:
# 予測結果の結合
test_data["予測結果"] = pred

In [461]:
# 予測結果をint型に変換
test_data["予測結果"] = test_data["予測結果"].astype(int)

In [463]:
result = test_data[["id", "予測結果"]]

In [464]:
result

,id,予測結果
0,31471,158098
1,31472,80236
2,31473,94671
3,31474,69022
4,31475,133378
...,...,...
31257,62728,103650
31258,62729,103275
31259,62730,176716
31260,62731,262251


In [470]:
result.to_csv("result.csv", header=False, index=False)